In [ ]:
import numpy as np
import os.path as osp
import os
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 
import sys
from scipy.sparse import lil_matrix

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

from hidden_points_removal import hidden_point_removal_o3d, hidden_point_removal_biasutti
from point_cloud_utils import transform_pcd, filter_points_from_dict, get_pcd, point_to_label, change_point_indices
from point_to_pixels import point_to_pixel
from visualization_utils import unite_pcd_and_img, color_pcd_with_labels, visualize_associations_in_img
from merge_pointclouds import build_associations, apply_associations_to_dict, merge_label_predictions, merge_pointclouds, build_associations_across_timesteps
from merged_sequences import merged_sequence
from reproject_merged_pointcloud import reproject_merged_pointcloud

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/laurenzheidrich/Downloads/','fused_dataset')
SEQUENCE_NUM = 7

config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    correct_scan_calibration=True,
    filters=FilterList(
        [
            KittiGTMovingObjectFilter(
                os.path.join(
                    DATASET_PATH,
                    "sequences",
                    "%.2d" % SEQUENCE_NUM,
                    "labels",
                )
            ),
            RangeFilter(2.5, 120),
            ApplyPose(),
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

Now we read in the point cloud and the left and right image of the stereo camera. If labels for those images are available they can be read in, too!

In [ ]:
start_index = 58
sequence_length = 10

label_history, pcd_merge = merged_sequence(dataset, start_index, sequence_length, labelling=False)

In [ ]:
feature_matrix = reproject_merged_pointcloud(np.array(pcd_merge.points), dataset, start_index, sequence_length)

In [ ]:
from sklearn.decomposition import TruncatedSVD
n_components = 75
tSVD = TruncatedSVD(n_components=n_components)
transformed_data = tSVD.fit_transform(feature_matrix)

In [ ]:
from sklearn.cluster import Birch, KMeans, MeanShift, DBSCAN

# Birch is fastest, results are also quite nice
birch_model = Birch(threshold=0.1, n_clusters=100)
birch_model.fit(transformed_data)
labels = birch_model.predict(transformed_data)

# kmeans is quite fast, results look okay, some noise
#kmeans = KMeans(n_clusters=80)
#labels = kmeans.fit_predict(transformed_data)

# DBSCAN doesn't really produce great results and takes really long
#dbscan = DBSCAN(eps=0.5, min_samples=100)
#labels = dbscan.fit_predict(transformed_data)

# Meanshift takes way too long
#meanshift = MeanShift()
#labels = meanshift.fit_predict(transformed_data)

In [ ]:
print("Cluster Assignments:")
print(len(set(labels)))

In [ ]:
colorspace = plt.cm.rainbow(np.linspace(0, 1, len(set(labels))))[:, :3]
colors = [colorspace[i] for i in labels]
pcd_merge.colors = o3d.utility.Vector3dVector(np.array(colors))

In [ ]:
#o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd_merge, write_ascii=False, compressed=False, print_progress=False)